In [0]:

import duckdb
import s3fs
import pandas as pd
from datetime import datetime

# ----------------------------
# 1) Conexão DuckDB e extensão S3
# ----------------------------
con = duckdb.connect()

con.execute("INSTALL httpfs;")
con.execute("LOAD httpfs;")

# ----------------------------
# 2) Configuração das credenciais AWS
# ----------------------------
con.execute("SET s3_region='us-east-1';")
con.execute("SET s3_access_key_id='AWS_ACCESS_KEY_ID';")
con.execute("SET s3_secret_access_key='AWS_SECRET_ACCESS_KEY';")


# ----------------------------
# 3) Caminho da Silver (ajustado com nome real do arquivo)
# ----------------------------
silver_path = "s3://nasa-weather-bronze/silver/nasa_weather/nasa_insight_silver_2025-11-05.parquet"

# ----------------------------
# 4) Leitura da Silver e criação da tabela
# ----------------------------
con.execute(f"""
CREATE OR REPLACE TABLE silver_data AS
SELECT * FROM read_parquet('{silver_path}');
""")
print("✅ Tabela silver_data criada com sucesso.")

# ----------------------------
# 5) Criação da tabela GOLD por estação
# ----------------------------
con.execute("""
CREATE OR REPLACE TABLE gold_by_season AS
SELECT
    season,
    AVG(temp_celsius) AS avg_temp_c,
    MIN(temp_celsius) AS min_temp_c,
    MAX(temp_celsius) AS max_temp_c,
    STDDEV(temp_celsius) AS stddev_temp_c,
    COUNT(*) AS observations
FROM silver_data
GROUP BY season
ORDER BY avg_temp_c DESC;
""")
print("✅ Tabela gold_by_season criada com sucesso.")

# ----------------------------
# 6) Exportação do GOLD para o S3
# ----------------------------
data_str = datetime.now().strftime("%Y-%m-%d")
gold_path = f"s3://nasa-weather-bronze/gold/nasa_weather/parquet/nasa_gold_{data_str}.parquet"

fs = s3fs.S3FileSystem(
   aws_access_key_id = dbutils.secrets.get("aws", "access_key_id")
aws_secret_access_key = dbutils.secrets.get("aws", "secret_access_key")
    client_kwargs={'region_name': 'us-east-1'}
)

df_gold = con.execute("SELECT * FROM gold_by_season").fetchdf()
with fs.open(gold_path, 'wb') as f:
    df_gold.to_parquet(f, index=False)

print("✅ GOLD exportado com sucesso para:", gold_path)
